In [25]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/naomiverkerk/TM10007.git


In [69]:
## Import
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm
from sklearn import decomposition
from load_data import load_data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Classifiers and kernels
import numpy as np
from numpy import reshape
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [70]:
##Loading Data
data = load_data() 
X = data
X = pd.DataFrame(X)
X = X.replace(np.inf, np.nan)
Y = data['label']
Y = pd.DataFrame(Y)
del X['label']

In [71]:
## Split
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size = 0.2, random_state = 4, stratify = Y)

X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)
X_test = pd.DataFrame(X_train)
Y_test = pd.DataFrame (Y_test)

In [72]:
## Features weghalen met teveel missing values
acceptabele_ratio = 0.5
train_size = len(X_train.index)
removal_rate = round(train_size*acceptabele_ratio)

X_train = pd.DataFrame(X_train)
X_train = X_train.dropna(axis=1, thresh=removal_rate)
common_cols = list(set(X_train.columns).intersection(X_test.columns))
X_test = X_test[common_cols]

In [73]:
## Imputation met median
X_train_missing_median = X_train.fillna(X_train.median())
X_train_missing_median = np.nan_to_num(X_train_missing_median)
X_test_missing_median = X_test.fillna(X_test.median())
X_test_missing_median = np.nan_to_num(X_test_missing_median)

In [74]:
## Scaling
scaler = preprocessing.RobustScaler()
scaler.fit(X_train_missing_median)
X_train_scaled = scaler.transform(X_train_missing_median)
X_train_scaled = np.nan_to_num(X_train_scaled)
X_test_scaled = scaler.transform(X_test_missing_median)
X_test_scaled = np.nan_to_num(X_test_scaled)

In [75]:
## PCA
pca = decomposition.PCA(n_components=0.99, svd_solver= 'full')
pca.fit(X_train_scaled)
X_train_final = pca.transform(X_train_scaled)
X_test_final = pca.transform(X_test_scaled)
explained_variance = np.cumsum(pca.explained_variance_ratio_)


In [81]:
# Construct different SVM methods
model= SVC()
model.fit(X_train_final, Y_train)

param_grid = {'degree': [1, 3, 5],
              'coef0': [0.01, 0.5, 1],
              'C': [0.01, 0.5, 1],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ('linear', 'poly', 'rbf', 'sigmoid')}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)


Y_array = np.asarray(Y_train)
Y_train_final = Y_array.reshape((133,1))



C:\Users\nwver\miniconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [82]:

for X, Y in zip(np.transpose(X_train_final), np.transpose(Y_train_final)):
    # Fit the grid
    grid.fit(X, Y)


# Print best parameter after tuning
print(grid.best_params_)

# Print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

# Predict with the trained model
grid_predictions = grid.predict(X_test_final)

# Print calssification report
print(classification_report(Y_test, grid_predictions))


Fitting 5 folds for each of 432 candidates, totalling 2160 fits
[CV 1/5] END C=0.01, coef0=0.01, degree=1, gamma=1, kernel=linear;, score=nan total time=   0.0s
[CV 2/5] END C=0.01, coef0=0.01, degree=1, gamma=1, kernel=linear;, score=nan total time=   0.0s
[CV 3/5] END C=0.01, coef0=0.01, degree=1, gamma=1, kernel=linear;, score=nan total time=   0.0s
[CV 4/5] END C=0.01, coef0=0.01, degree=1, gamma=1, kernel=linear;, score=nan total time=   0.0s
[CV 5/5] END C=0.01, coef0=0.01, degree=1, gamma=1, kernel=linear;, score=nan total time=   0.0s
[CV 1/5] END C=0.01, coef0=0.01, degree=1, gamma=1, kernel=poly;, score=nan total time=   0.0s
[CV 2/5] END C=0.01, coef0=0.01, degree=1, gamma=1, kernel=poly;, score=nan total time=   0.0s
[CV 3/5] END C=0.01, coef0=0.01, degree=1, gamma=1, kernel=poly;, score=nan total time=   0.0s
[CV 4/5] END C=0.01, coef0=0.01, degree=1, gamma=1, kernel=poly;, score=nan total time=   0.0s
[CV 5/5] END C=0.01, coef0=0.01, degree=1, gamma=1, kernel=poly;, score

C:\Users\nwver\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
2160 fits failed out of a total of 2160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
432 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\nwver\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nwver\miniconda3\lib\site-packages\sklearn\svm\_base.py", line 190, in fit
    X, y = self._validate_data(
  File "C:\Users\nwver\miniconda3\lib\site-packages\sklearn\base.py", line 581, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\nwver\miniconda3\lib\

ValueError: Expected 2D array, got 1D array instead:
array=[-1.43620781e+02 -1.43559913e+02 -1.43755989e+02 -1.25843621e+02
 -1.43613115e+02 -5.92054011e+01 -1.43603799e+02 -1.43599022e+02
  1.56667746e+03 -1.38510839e+02 -1.44475370e+02 -1.44421727e+02
 -1.48388181e+02 -1.43561025e+02 -1.43679099e+02 -1.43303005e+02
  3.10095810e+03 -1.43581487e+02 -1.38772391e+02 -1.43669978e+02
 -1.43874050e+02 -1.43586576e+02 -1.43634540e+02 -1.43645331e+02
 -1.43645539e+02 -1.43615025e+02  7.01046186e+01 -1.43628994e+02
 -1.42939047e+02 -1.43741135e+02 -1.43754927e+02 -7.76716761e+01
 -7.89659742e+01 -2.42111764e+01 -1.43758883e+02 -1.46434954e+01
 -1.43590835e+02 -1.43712744e+02 -1.44287486e+02 -1.43919754e+02
 -4.34115419e+01 -1.43544145e+02 -1.43799300e+02  3.83420490e+03
 -1.43585930e+02 -1.45826744e+02 -1.43538756e+02 -1.44117801e+02
 -1.17512036e+02 -1.43916792e+02 -1.43737454e+02 -1.38778175e+02
 -1.43909338e+02 -1.44800310e+02  1.29727419e+02 -1.43620921e+02
 -1.43569604e+02 -1.43569960e+0

In [ ]:
# Construct different Random Forest methods
n_estimators = [1, 5, 10, 25, 50, 75, 100]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(4, 80, 4)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
class_weight = ['balanced', 'balanced_subsample']

param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap,
              'class_weight': class_weight}

grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3)

# Fit the grid
grid.fit(X_train, Y_train)

# Print best parameter after tuning
print(grid.best_params_)

# Print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

# Predict with the trained model
grid_predictions = grid.predict(X_test)

# Print calssification report
print(classification_report(Y_test, grid_predictions))
